The code in this notebook is based off the two notebooks: 
* 4-7-RQ4-BERTScores
* 4-4-UserStudy2_preference_hierarchical_sim

# Loading data and necessary files

In [ ]:
from bert_score import BERTScorer
from collections import defaultdict
import itertools
import numpy as np
import os
import pandas as pd
import re
from src.config import DATA_DIR, TEMP_DIR
from src.config import label_file, label_file_full, weighing_scheme
from src.config import BERT_F1_dict_file
from src.io import load_pickle, save_pickle
import time

# Global variables:
# name_df, gt_df, rec_df
# models

name_df = pd.read_csv(os.path.join(TEMP_DIR, 'UserStudy1', 'full_name_ref.csv'))
gt_df = pd.read_csv(os.path.join(TEMP_DIR, 'UserStudy1', 'user_ground_truth.csv')).drop_duplicates().reset_index(drop=True)
rec_df = pd.read_csv(os.path.join(TEMP_DIR, 'UserStudy1', 'user_recommendations.csv'))
rec_df = rec_df.rename(columns={"item_idx": "PID"})

gt_df = gt_df.sort_values(by=['UID', 'PID'])
rec_df = rec_df.sort_values(by=['UID', 'PID'])

gt_df = pd.merge(gt_df, name_df, how='left', on=['PID'])
gt_df = gt_df.rename(columns={"name": "gt_item"})
rec_df = pd.merge(rec_df, name_df, how='left', on=['PID'])
rec_df = rec_df.rename(columns={"name": "rec_item"})

models = sorted(set(rec_df['models'].values))

In [ ]:
# Global variables:
# sim_dict
# scorer

if os.path.exists(BERT_F1_dict_file):
    sim_dict = load_pickle(BERT_F1_dict_file)
else:
    sim_dict = {}

loaded_sim_dict_len = len(sim_dict)
print('Loaded pre-computed BERTScores for tags, %s total records' % loaded_sim_dict_len)

import time
start = time.time()
scorer = BERTScorer(lang="en", rescale_with_baseline=True, device="cuda")
end = time.time()
print(end - start)

In [ ]:
print(models)

In [ ]:
print(name_df.columns)
print(gt_df.columns)
print(rec_df.columns)

In [ ]:
name_df.nunique()

In [ ]:
gt_df.nunique()

In [ ]:
rec_df.nunique()

# Data Cleaning
Some models, i.e., adaloyal, fpmc, and personal, predicted more than 1 item for certain ranks for certain users (N=19526). In that case, we will randomly select one item to represet each nth-rank item.

In [ ]:
print(rec_df.shape)
count_df = rec_df.groupby(['UID','models','pred_rank']).count().sort_values(by=['rec_item'], ascending=False)
count_df = count_df.reset_index()
count_df[count_df['PID']>1]

In [ ]:
start = time.time()
rec_df = rec_df.groupby(['UID', 'models', 'pred_rank']).sample()
end = time.time()
print(rec_df.shape)
print('%.2f sec' % (end-start))

In [ ]:
rand_state = 100 # random state. 100 in the main experiment.
rand_df = rec_df[rec_df['models']=='global'] # use global as a template. But any model will do.
rand_df['models'] = 'random'
rand_df['rand_item'] = list(name_df['name'].sample(n = rand_df.shape[0], replace=True, random_state = rand_state)) # sample from all possible food items stored in name_df
rand_df = rand_df.drop('rec_item', axis=1)
rand_df = rand_df.rename(columns={'rand_item': 'rec_item'})
rec_df = pd.concat([rec_df, rand_df], ignore_index = True)

In [ ]:
def process_food_name(s1):
    # separators: ", " + any of (integer, decimal & fraction) +" "
    exp = r", \d+\.\d+ |, \d+\,\d+ |, \d+ |, \d+\/\d+ "
    # remove content in parenthesis for finding the separator
    if s1.count('(') == s1.count(')'):
        s2 = re.sub(r'[(].*?[\)]', ' ', s1)
    else:
        s2 = s1
    try:
        split_by = re.findall(exp, s2)[0]
        return clean_name(s1.split(split_by)[0])
    except:
        return clean_name(s2)

def clean_name(name):
    name = name.replace("\t", " ").replace("\n", " ").replace("w/o", " no ").replace("w/", " ")
    return re.sub(' +', ' ', name.strip()).lower()

Process all food names by removing puctuations and numerals

Mapping between variable names (this notebook's, the reference notebook's):

For (gt_df, df2):
* (UID, user)
* (gt_item, item_1)
* (gt_item_cleaned, item_gt)

For (rec_df, df):
* (UID, user)
* (rec_item, choice)
* (rec_item_cleaned, item)
* (models, qn)

In [ ]:
rec_df['rec_item_cleaned'] = rec_df['rec_item'].apply(process_food_name)
rec_df = rec_df.rename(columns={'UID': 'user', 'rec_item': 'choice', 'rec_item_cleaned': 'item', 'models': 'qn'})
gt_df['gt_item_cleaned'] = gt_df['gt_item'].apply(process_food_name)
gt_df = gt_df.rename(columns={'UID': 'user', 'gt_item': 'item_1', 'gt_item_cleaned': 'item_gt'})

In [ ]:
labels = load_pickle(label_file)
concat_list = list(zip(labels['cat_info'], labels['label_summary']))
label_index = dict()
for i,j in concat_list:
    label_index.update(zip(i,j))  

index_label = {k:v for v,k in label_index.items()}

# '_' connected tokens for l0 tags
label_index_l0 = {i.split('__')[-1]:v for i,v in label_index.items() if len(i.split('__'))==3}
l0_tags = sorted(label_index_l0.keys())

label_with_food = defaultdict(list)
def group_food_name(line):
    s = line['food_name']
    for i in line['label_summary']:
        label_with_food[i].append(s)
labels.apply(group_food_name, axis=1) 

label_name_l0 = {}
for l,i in label_index_l0.items():
    label_name_l0[l] = label_with_food[i]

name_labels = labels.set_index('food_name').to_dict()['cat_info']
name_label_0 = {k:[s.split('__')[-1] for s in v if len(s.split('__'))==3] for k,v in name_labels.items()}

matched_label_l0 = []
unmatched_label_l0 = []

def token_transform(t):
    tokens = [t, t+'s', t+'es']
    if t[-1] == 'y':
        tokens.append(t[:-1]+'ies')
    return tokens  

for l, s_lst in label_name_l0.items():    
    l_primes = [token_transform(t) for t in l.split('_')]
    l2 = list(itertools.product(*l_primes))
    l_primes = [' '.join(i) for i in l2]  
 
    matched = False
    for s in s_lst:
        s = ' '.join(s.replace("'", '').replace('&', ' ').split())
        for l_prime in l_primes:
            if l_prime in s:            
                matched = True
            
    if matched:
        matched_label_l0.append(l)
    else:
        unmatched_label_l0.append(l)  

perc = len(unmatched_label_l0)/(len(matched_label_l0 )+ len(unmatched_label_l0)) 
if perc < 0.05:
    print('Less than 5% labels are not matched: {:.2%}'.format(perc))

# get all labels associated with item
def match_labels(s):
    if s in name_label_0.keys():
        found = name_label_0[s] 
    else:
        found = []
        s = ' '.join(s.replace("'", '').replace('&', ' ').split())
        for l in matched_label_l0:
            matched = False
            l_primes = [token_transform(t) for t in l.split('_')]
            for l_prime in [' '.join(i) for i in list(itertools.product(*l_primes))]:
                if l_prime in s:  
                    matched = True
            if matched:
                found.append(l) 
                        
    # full label names
    all_labels = []
    if len(found) > 0:
        full_label_l0 = [index_label[label_index_l0[l0_label]] for l0_label in found]
        for l0 in full_label_l0:
            l2 = l0.split('__')[0]
            l1 = '__'.join(l0.split('__')[:2])
            all_labels.extend([l2, l1, l0])
    return sorted(set(all_labels))

# Two variants: 1) using all tags along the branch; 2) using only the last tag
inv_label_index = {v: " ".join(k.replace("__", " ").split("_")) for k, v in label_index.items()} # use all tags along the branch
# inv_label_index = {v: " ".join(k.split("__")[-1].split("_")) for k, v in label_index.items()} # use only the last tag

In [ ]:
rec_df.loc[:, 'label']  = rec_df['item'].apply(match_labels)
rec_df['label_summary'] = rec_df['label'].apply(lambda s: [label_index[i] for i in s] if len(s)>0 else [])

gt_df.loc[:, 'label']  = gt_df['item_gt'].apply(match_labels)
gt_df['label_summary'] = gt_df['label'].apply(lambda s: [label_index[i] for i in s] if len(s)>0 else [])
gt_df = gt_df[['user', 'item_1', 'label', 'label_summary']]

gt_lst = gt_df.groupby('user')['label_summary'].apply(list).to_dict()
rec_df['l2'] = rec_df['user'].map(gt_lst)

df1 = rec_df.groupby(['user', 'qn'])['label_summary'].apply(list).reset_index().rename(columns={'label_summary': 'l2'})
df1 = gt_df.merge(df1, on='user', how='outer')

# Scoring functions

In [ ]:
def get_scores(single_cands, multi_refs):
    P_mul, R_mul, F_mul = scorer.score([single_cands], [multi_refs])
    return F_mul[0]

def sim_score(l0, l1, sim_dict=sim_dict):
    l0, l1 = tuple(sorted([l0, l1]))
    
    if (l0, l1) in sim_dict.keys():
        return sim_dict[(l0, l1)]
    else:
        if l0 == l1: 
            val = 1
        else:
            val = get_scores(l0.lower(), l1.lower())
        sim_dict[(l0, l1)] = val
        return float(val)
    
def label_score(c_cap_j, c_cap_j_prime, weight, sim_dict=sim_dict):
    if len(c_cap_j)==0 or len(c_cap_j_prime)==0:
        return 0
        #return np.nan
    
    numerator = 0.0
    denominator = 0.0

    for c_t in c_cap_j:
        lambda_t = weight[c_t]
        denominator += lambda_t
        max_val = -1
       
        for c_s in c_cap_j_prime:
            val = sim_score(inv_label_index[c_t], inv_label_index[c_s], sim_dict)
            if val > max_val:
                max_val = val
        
        numerator += lambda_t * max_val
    
    return numerator / denominator

def pair_sim_scores(j, j_prime, weight, sim_dict):
    if(len(j)==0) or (len(j_prime)==0):
        return 0
    
    if j_prime == j:
        return 1
    else:
        curr_score = label_score(j, j_prime, weight, sim_dict)
    return float(curr_score)

In [ ]:
x = 'Egg Salad Sandwich'
y = 'dreyers grand ice cream - chocolate ice cream'
x = 'Home Made - Garden Salad'
y = 'chips - chips'
y = 'coke - coke (330ml)'
c_cap_j = match_labels(x)
c_cap_j_prime = match_labels(y)
# print(c_cap_j)
# print(c_cap_j_prime)
score = None

if len(c_cap_j)==0 or len(c_cap_j_prime)==0:
    score = 0

# print(score)
# print(labels[labels['food_name']==x.lower()]['label_summary'])
# print(labels[labels['food_name']==y.lower()]['label_summary'])
print(name_label_0[y.lower()])

# Benchmarking

To compute hMatch_sim from scratch, it takes about (without GPU):
* To be completed

With GPU:
* 1 min for about 32 rows (10 users)
* 2 min for 800 rows
* 30 min for 28,909 rows or one model (6915 users)

# Main

Compute hMatch_sim for each model. Data will be split into batches so that results can be saved to CSV periodically.

In [ ]:
start = time.time()
# 'adaloyal', 'bpr', 'fpmc', 'global', 'lda', 'mixture', 'mixture_decay', 'nmf', 'personal', 'sasrec', 'wrmf', 'random'
m = 'random'
start_uid = 0
batch_size = 200
max_uid = 7000
weighing='freq'
col = 'hSim-freq'

def lst_sim_scores(line, weight=weighing_scheme[weighing], sim_dict=sim_dict):
    j = line['label_summary']
    j_primes = line['l2']
    pair_sim_score = 0
    for j_prime in j_primes:
        s = pair_sim_scores(j, j_prime, weight, sim_dict)
        if s > pair_sim_score:
            pair_sim_score = s
    return pair_sim_score

# We select each batch based on UID and models. batch_size=200 normally results in ~9000 rows of data for computing per batch.
while start_uid < max_uid:
    istart = time.time()
    end_uid = start_uid + batch_size  
    
    eval_df = df1[(df1['user']>=start_uid) & (df1['user']<end_uid) & (df1['qn']==m)] # Selecting subset of data
    # Calculating BERTScores, will take quite some time if on a non-CUDA machine
    eval_df[col] = eval_df.apply(lst_sim_scores, axis=1)
    
    iend = time.time()
    eval_df.to_csv('data/temp/hmatch_sim/%s_uid_%s.csv' % (m, start_uid), index=False)
    print('total rows=%s\tstart_uid=%s\tend_uid=%s\telapsed_time=%.2f min' % (len(eval_df), start_uid, end_uid, (iend - istart)/60))
    start_uid += batch_size
    time.sleep(1)

end = time.time()
print('total_time=%.2f min' % ((end - start)/60))

Join all results into one file

In [ ]:
# concatenating multiple results into one
m = 'random'
start_uid = 0
batch_size = 200
max_uid = 7000

big_df = pd.DataFrame()

while start_uid < max_uid:
    try:
        df = pd.read_csv('data/temp/hmatch_sim/%s_uid_%s.csv' % (m, start_uid))
    except FileNotFoundError:
        print('data/temp/hmatch_sim/%s_uid_%s.csv does not exist.' % (m, start_uid))
    
    big_df = pd.concat([big_df, df], ignore_index=True)
    start_uid += batch_size

big_df.to_csv('data/temp/hmatch_sim/%s.csv' % (m), index=False)

Computing aggregated top-k scores for each user and final scores for the model

In [ ]:
start = time.time()
user_mean_df = big_df.reset_index().groupby(['user'])[['hSim-freq']].mean()
end = time.time()
print('%.2f sec' % (end-start))
print(m)
print(user_mean_df.mean())

Persisting computed scores to a pickle file

In [ ]:
print(len(sim_dict))
# for k, v in sim_dict.items():
#     print('%s\t%s\t%.2f\t%.2f\t%.2f' % (k[0], k[1], v['P'], v['R'], v['F']))

In [ ]:
if len(sim_dict)>=loaded_sim_dict_len:
    save_pickle(BERT_F1_dict_file, sim_dict)
    print('Saved pre-computed scores, %s total records' % len(sim_dict))